## 4.8 Grouping & Aggregating 

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import os

In [4]:
#Importing Data Frame
path=r"C:\Users\danac\Downloads\Instacart Basket Analysis"

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged2.pkl'))

In [5]:
ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])

## point 2 : Finding the "mean" in the subset and in the whole dataframe

In [10]:
#Creating the Subset 
ords_prods_merge_sub=ords_prods_merge[:1000000]

In [12]:
#Grouping and aggregating
ords_prods_merge_sub.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.577493
2,17.320781
3,16.084944
4,17.530458
5,14.763075
6,16.658449
7,17.031590
8,15.076662


In [14]:
#Grouping and aggregating in the whole dataframe
ords_prods_merge.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Explanation: comparing the results from the subset against the whole dataframe, what i can see is that there is not that much difference in the "mean" of both Dataframe, just small variations.

## Point 4 : Creating the Loyalty Flag

In [19]:
# Creating new column "max_order"
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\danac\AppData\Local\Temp\ipykernel_20352\2324063168.py:2: FutureWarning: The provided callable <function max at 0x000001FBD202C540> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [21]:
# Checking the new column
ords_prods_merge.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,add_to_cart_order,reordered,price_range_loc,busiest_day,busiests_day,Busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,Mid-range product,Regularly busy,Regularly busy,Average Orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,Mid-range product,Regularly busy,Regularly busy,Average Orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,Mid-range product,Busiest day,Busiests day,Average Orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,Mid-range product,Regularly busy,Leasts busy,Average Orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,Mid-range product,Least busy,Leasts busy,Average Orders,3


In [23]:
#Creating the  loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [25]:
#Checking the loyalty flag
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [27]:
#Checking the loyalty flag in dataframe
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number','max_order']].head(5)

,user_id,loyalty_flag,order_number,max_order
0,138,Regular customer,28,32
1,138,Regular customer,30,32
2,709,New customer,2,5
3,764,New customer,1,3
4,764,New customer,3,3


## Point 5 : Basic statistics of the Loyalty flag column

In [30]:
#The min max and mean of each loyalty flag
ords_prods_merge.groupby(['loyalty_flag']).agg({'prices':['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

## Point 6 : Creating a Spending Flag based on the average price

In [34]:
#Creating the column for Average spender
ords_prods_merge['average_spend']=ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\danac\AppData\Local\Temp\ipykernel_20352\2391603878.py:2: FutureWarning: The provided callable <function mean at 0x000001FBD202CF40> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_spend']=ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [36]:
#Creating the Spender flag for low and High spender
ords_prods_merge.loc[ords_prods_merge['average_spend']<10,'spender_flag']='Low_spender'

ords_prods_merge.loc[ords_prods_merge['average_spend']>= 10, 'spender_flag'] = 'High_spender'

In [38]:
#Checking the Spender flag
ords_prods_merge['spender_flag'].value_counts(dropna = False)

spender_flag
Low_spender     31770614
High_spender      634245
Name: count, dtype: int64

In [42]:
#Checking in the dataframe
ords_prods_merge[['user_id', 'average_spend', 'spender_flag', 'prices']].head(5)

,user_id,average_spend,spender_flag,prices
0,138,6.935811,Low_spender,5.8
1,138,6.935811,Low_spender,5.8
2,709,7.930208,Low_spender,5.8
3,764,4.972414,Low_spender,5.8
4,764,4.972414,Low_spender,5.8


## Point 7 : Creating an order Frecuency flag

In [47]:
# Creating a column for frecuent users
ords_prods_merge['Customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

C:\Users\danac\AppData\Local\Temp\ipykernel_20352\3277188001.py:2: FutureWarning: The provided callable <function median at 0x000001FBD214BBA0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['Customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)


In [49]:
# Creating the Frecuency flag 
ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] > 20, 'frequency_flag'] = 'Non_frequent_customer'

ords_prods_merge.loc[(ords_prods_merge['Customer_frequency']>10) & (ords_prods_merge['Customer_frequency']<=20),'frequency_flag'] = 'Regular customer'

ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [51]:
# Checking the Frequency flags
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        21559853
Regular customer          7208564
Non_frequent_customer     3636437
NaN                             5
Name: count, dtype: int64

In [53]:
#Checking in the Dataframe
ords_prods_merge[['user_id','days_since_last_order', 'Customer_frequency', 'frequency_flag']].head(5)

,user_id,days_since_last_order,Customer_frequency,frequency_flag
0,138,3.0,8.0,Frequent Customer
1,138,20.0,8.0,Frequent Customer
2,709,6.0,8.0,Frequent Customer
3,764,NaN,9.0,Frequent Customer
4,764,9.0,9.0,Frequent Customer


## Exporting the Dataframe

In [56]:
ords_prods_merge.to_pickle(r'C:\Users\danac\Downloads\Instacart Basket Analysis\02 Data\Prepared Data\orders_products_aggregated.pkl')